In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'E:/data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 128
beta = 0.002
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  l2_reg = tf.nn.l2_loss(weights)
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss += beta * l2_reg
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 23.391048
Minibatch accuracy: 9.4%
Validation accuracy: 14.6%
Minibatch loss at step 500: 2.650759
Minibatch accuracy: 82.8%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 1.313712
Minibatch accuracy: 79.7%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 0.679150
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.621022
Minibatch accuracy: 89.1%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.721103
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.703992
Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Test accuracy: 88.9%


In [7]:
#adding L2 regularization
batch_size = 128
beta = 0.0014
graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    hidden_layer1_size = 1024
    
    hidden1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size]))
    hidden1_biases=tf.Variable(tf.zeros([hidden_layer1_size]))
    hidden1_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)   
  
    output_weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden1_layer, output_weights) + output_biases
    l2_reg = tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(hidden1_weights)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * l2_reg
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)

  # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)     
    valid_logits = tf.matmul(valid_hidden1, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

  # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)      
    test_logits = tf.matmul(test_hidden1, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 742.41
Minibatch accuracy: 15.6%
Validation accuracy: 32.1%
Minibatch loss at step 500 : 223.245
Minibatch accuracy: 80.5%
Validation accuracy: 79.0%
Minibatch loss at step 1000 : 107.29
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 1500 : 52.5982
Minibatch accuracy: 88.3%
Validation accuracy: 84.5%
Minibatch loss at step 2000 : 26.1502
Minibatch accuracy: 90.6%
Validation accuracy: 85.7%
Minibatch loss at step 2500 : 13.249
Minibatch accuracy: 89.1%
Validation accuracy: 86.4%
Minibatch loss at step 3000 : 6.90749
Minibatch accuracy: 85.2%
Validation accuracy: 86.7%
Test accuracy: 93.1%


In [9]:
#adding a dropout
batch_size = 128
beta = 0.0014
graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    hidden_layer1_size = 1024
    
    hidden1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size]))
    hidden1_biases=tf.Variable(tf.zeros([hidden_layer1_size]))
    hidden1_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases), 0.6)

    output_weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden1_layer, output_weights) + output_biases
    l2_reg = tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(hidden1_weights)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * l2_reg
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)

  # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)     
    valid_logits = tf.matmul(valid_hidden1, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

  # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)      
    test_logits = tf.matmul(test_hidden1, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)


In [10]:
num_steps = 16001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 833.223
Minibatch accuracy: 10.9%
Validation accuracy: 31.9%
Minibatch loss at step 500 : 229.567
Minibatch accuracy: 74.2%
Validation accuracy: 79.5%
Minibatch loss at step 1000 : 107.349
Minibatch accuracy: 78.9%
Validation accuracy: 81.4%
Minibatch loss at step 1500 : 52.4625
Minibatch accuracy: 85.9%
Validation accuracy: 83.3%
Minibatch loss at step 2000 : 26.0289
Minibatch accuracy: 90.6%
Validation accuracy: 85.1%
Minibatch loss at step 2500 : 13.2268
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Minibatch loss at step 3000 : 6.9068
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
Minibatch loss at step 3500 : 3.8101
Minibatch accuracy: 83.6%
Validation accuracy: 86.2%
Minibatch loss at step 4000 : 2.10688
Minibatch accuracy: 89.8%
Validation accuracy: 86.6%
Minibatch loss at step 4500 : 1.36355
Minibatch accuracy: 85.9%
Validation accuracy: 86.6%
Minibatch loss at step 5000 : 0.958728
Minibatch accuracy: 85.9%
Validation accuracy

In [50]:
#adding a second layer for improving
batch_size = 256
beta = 0.0005
graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    hidden_layer1_size = 1024
    hidden_layer2_size = 300
    
    hidden1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size]))
    hidden1_biases=tf.Variable(tf.zeros([hidden_layer1_size]))
    hidden1_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases), 0.8)
    
    hidden2_weights = tf.Variable(
    tf.truncated_normal([hidden_layer1_size, hidden_layer2_size]))
    hidden2_biases=tf.Variable(tf.zeros([hidden_layer2_size]))
    hidden2_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1_layer, hidden2_weights) + hidden2_biases), 0.8)

    output_weights = tf.Variable(
    tf.truncated_normal([300, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden2_layer, output_weights) + output_biases
    l2_reg = tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(hidden1_weights) + tf.nn.l2_loss(hidden2_weights)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * l2_reg
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.1, global_step, 7000, 0.96, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    train_prediction = tf.nn.softmax(logits)

  # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, hidden2_weights) + hidden2_biases)    
    valid_logits = tf.matmul(valid_hidden2, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

  # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, hidden2_weights) + hidden2_biases)  
    test_logits = tf.matmul(test_hidden2, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)


In [51]:
num_steps = 78001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 4779.35
Minibatch accuracy: 10.5%
Validation accuracy: 21.4%
Minibatch loss at step 500 : 440.814
Minibatch accuracy: 13.3%
Validation accuracy: 10.4%
Minibatch loss at step 1000 : 419.399
Minibatch accuracy: 12.9%
Validation accuracy: 11.5%
Minibatch loss at step 1500 : 399.041
Minibatch accuracy: 12.9%
Validation accuracy: 12.1%
Minibatch loss at step 2000 : 379.703
Minibatch accuracy: 7.8%
Validation accuracy: 12.1%
Minibatch loss at step 2500 : 361.313
Minibatch accuracy: 11.7%
Validation accuracy: 13.8%
Minibatch loss at step 3000 : 343.792
Minibatch accuracy: 9.0%
Validation accuracy: 12.8%
Minibatch loss at step 3500 : 327.046
Minibatch accuracy: 14.1%
Validation accuracy: 14.4%
Minibatch loss at step 4000 : 311.229
Minibatch accuracy: 15.2%
Validation accuracy: 13.1%
Minibatch loss at step 4500 : 296.154
Minibatch accuracy: 10.9%
Validation accuracy: 15.0%
Minibatch loss at step 5000 : 281.795
Minibatch accuracy: 14.8%
Validation accuracy:

Minibatch loss at step 45000 : 8.65218
Minibatch accuracy: 82.4%
Validation accuracy: 85.7%
Minibatch loss at step 45500 : 8.26276
Minibatch accuracy: 86.3%
Validation accuracy: 86.0%
Minibatch loss at step 46000 : 8.16197
Minibatch accuracy: 80.9%
Validation accuracy: 85.8%
Minibatch loss at step 46500 : 7.89656
Minibatch accuracy: 82.8%
Validation accuracy: 85.9%
Minibatch loss at step 47000 : 7.41558
Minibatch accuracy: 85.5%
Validation accuracy: 86.2%
Minibatch loss at step 47500 : 7.27965
Minibatch accuracy: 78.9%
Validation accuracy: 86.3%
Minibatch loss at step 48000 : 6.94047
Minibatch accuracy: 84.0%
Validation accuracy: 86.3%
Minibatch loss at step 48500 : 6.6794
Minibatch accuracy: 84.8%
Validation accuracy: 86.4%
Minibatch loss at step 49000 : 6.44718
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Minibatch loss at step 49500 : 6.27411
Minibatch accuracy: 82.4%
Validation accuracy: 86.6%
Minibatch loss at step 50000 : 5.94805
Minibatch accuracy: 84.8%
Validation accur